### Workshop 6 - Use R to perform differential gene expression (DGE) analysis
  
1. Activate a [conda](https://www.anaconda.com/products/distribution) environment that has all the R libraries (packages) we need for the analysis (and much more)  
**Before running these commands make sure that the Kernel is changed to R!**

In [ ]:
old_path <- Sys.getenv("PATH")
new_path <- "/home/anaconda/miniconda3/bin"
Sys.setenv(PATH = paste(old_path, new_path, sep=":"))

2. Load the required R libraries/packages  

In [ ]:
options(warn=-1)
suppressMessages(library("DESeq2"))
# library("DESeq2")
library("ggplot2")
library("EnhancedVolcano")

3. Read the data files (containing read counts and sample metadata table)  

In [ ]:
directory <- "./HTSEQ/STAR37/"

sampleFiles <- list.files(directory)
sampleCondition <- sub("_\\d.tab","",sampleFiles)
samples <- data.frame(sampleName = sampleFiles,
                           run = sampleFiles,
                           condition = sampleCondition)
run <- as.vector(samples$run)
head(samples)

write.csv(as.data.frame(samples),
          file="output/Experimental_design.csv")


4. Process the data to normalise the read counts and perform PCA

In [ ]:
ddsHTSeq <- DESeqDataSetFromHTSeqCount(sampleTable = samples,
                                       directory = directory,
                                       design= ~ condition)
ddsHTSeq


keep <- rowSums(counts(ddsHTSeq)) >= 10
ddsHTSeq <- ddsHTSeq[keep,]

raw_counts <- data.frame(counts(ddsHTSeq))
write.csv(as.data.frame(raw_counts),
          file="output/Rawcounts.csv", col.names = TRUE)

vsd <- vst(ddsHTSeq, blind=FALSE)
head(assay(vsd), 3)

plotPCA(vsd, intgroup=c("condition")) + geom_text(aes(label=name), vjust=2)
ggsave("output/PCA.pdf", width=8, height=5)

write.csv(as.data.frame(assay(vsd)),
          file="output/Normalized_counts.csv")

5. Perform the DGE analysis and create a volcano plot 

In [ ]:
ddsHTSeq$condition <- relevel(ddsHTSeq$condition, ref = "Control")

ddsHTSeq <- DESeq(ddsHTSeq)
res <- results(ddsHTSeq)
res
resOrdered <- res[order(res$pvalue),]
resOrdered

write.csv(as.data.frame(resOrdered),
          file="output/DEG_Results.csv")


EnhancedVolcano(res,
                lab = rownames(res),
                x = 'log2FoldChange',
                y = 'pvalue',
                xlim = c(-5, 8))